### Assignment 1. Make the best model for TADA ETA data
Chapter 2에서 배운 tree algorithm들과 Chapter 3에서 배운 unsupervised learning을 통한 전처리를 응용해서 가장 좋은 성능을 내는 TADA ETA 예측모델을 만들어봅시다.

1. 트레이닝셋과 테스트셋의 split은 아래와 같이 고정하도록 하겠습니다. 실습 코드에서는 아래의 random_state=0이 없었어서 데이터셋 생성 시 마다 순서가 달라졌는데, 이번엔 모두가 같은 data split을 가지고 동등하게 평가를 진행하도록 하겠습니다. 
```
tada_eta = tada_eta.sample(frac=1, random_state=0).reset_index(drop=True)
train = tada_eta[:12000]
test = tada_eta[12000:]
```
2. input feature의 종류는 실습 코드에서는 4개만 사용했으나, 원 데이터셋에서 자유롭게 변형하여 사용하셔도 됩니다.
3. 구, 시간대, 월과 같은 feature를 예측에 더욱 더 효과적으로 활용하기 위해서 [data.seoul.go.kr](https://data.seoul.go.kr/dataList/datasetList.do)에 있는 서울시 공공데이터셋을 활용해도 됩니다. 아래 Reference에 사용가능한 데이터의 예시들이 있습니다. 다만 예측에 별로 도움이 되지 않는 데이터도 있을 수 있으니 잘 살펴보시고 사용해주세요.
4. 이미 구현된 코드 사용은 sklearn 패키지 안에 있는 tree, ensemble 모델과 unsupervised learning 알고리즘들로 제한하겠습니다. 다만 직접 구현하신다면 모델 종류에 제한을 두지 않겠습니다.
5. **채점 기준**: test set에 대한 MSE를 기준으로 순위를 매기도록 하겠습니다.


---

[Reference]
*   https://scikit-learn.org/stable/modules/tree.html
*   https://scikit-learn.org/stable/modules/ensemble.html
*  [서울시 차량통행속도 (구별/월별) 통계](https://data.seoul.go.kr/dataList/10846/S/2/datasetView.do)
*  [서울시 승용차통행속도 (시간대별) 통계](https://data.seoul.go.kr/dataList/265/S/2/datasetView.do)
*  [서울시 자동차등록 (월별/구별) 통계](https://data.seoul.go.kr/dataList/255/S/2/datasetView.do)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit

### Regression Tree with Tada ETA data



*   ATA: Actual Time of Arrival
*   ETA: Expected Time of Arrival
*   api_eta: 앱 API에서 예측된 ETA



In [ ]:
tada_eta = pd.read_excel('/content/drive/MyDrive/SOCARxLION/Chapter_2/Chapter 2_실습파일/tada_eta.xlsx')
print(tada_eta.columns)
tada_eta.head()

Index(['ATA', 'id', 'api_eta', 'created_at_kst', 'month', 'pickup_lng',
       'pickup_lat', 'pickup_gu', 'driver_id', 'driver_lng', 'driver_lat',
       'hour'],
      dtype='object')


,ATA,id,api_eta,created_at_kst,month,pickup_lng,pickup_lat,pickup_gu,driver_id,driver_lng,driver_lat,hour
0,10.60,R4L9FL6NVEL4D0,5.65,2019-12-01T00:55:09,12,127.091399,37.624545,노원구,DST52944,127.069978,37.616472,0
1,9.20,R3A7JBY7CS6TPD,8.60,2019-12-01T00:09:39,12,127.023351,37.625564,강북구,DPC72995,127.026479,37.604711,0
2,10.85,R2BIUY1TDK14XE,10.53,2019-12-15T00:18:42,12,126.881154,37.481641,금천구,DPP94923,126.906225,37.452167,0
3,5.52,R31PU5GNHSGU6C,1.92,2019-07-31T00:24:11,7,126.988375,37.567897,중구,DCT73585,126.991170,37.566416,0
4,8.60,R41WR6N1VT42DJ,7.10,2019-07-05T00:40:34,7,126.988684,37.566716,중구,DPJ86237,127.006860,37.571164,0


In [ ]:
tada_eta = tada_eta.sample(frac=1, random_state=0).reset_index(drop=True)

## tip 2


# Ordianl Encoding
enc = sklearn.preprocessing.OrdinalEncoder(dtype=np.int32)
ordinal = enc.fit_transform(np.asarray(tada_eta['pickup_gu']).reshape(-1,1))
tada_eta['pickup_gu'] = ordinal[:,0]
tada_eta['distance'] = ((tada_eta['pickup_lat']-tada_eta['driver_lat'])**2 + (tada_eta['pickup_lng']-tada_eta['driver_lng'])**2)*100000
tada_eta = tada_eta.drop(['id', 'created_at_kst', 'driver_id', 'pickup_lng', 'pickup_lat', 'driver_lng','driver_lat'],1)

tada_eta.head()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # This is added back by InteractiveShellApp.init_path()


,ATA,api_eta,month,pickup_gu,hour,distance
0,6.13,8.32,12,12,22,44.521977
1,5.37,3.05,12,17,11,3.989066
2,6.27,5.00,7,22,19,4.814699
3,10.52,8.47,12,20,21,77.341173
4,12.45,6.65,7,0,12,14.888851


In [ ]:
tada_eta.shape

(13916, 6)

In [ ]:
train = tada_eta[:12000]
test = tada_eta[12000:]

In [ ]:
# # normalization
# numeric_features=['api_eta', 'distance']  #'ATA'


# for num_feature in numeric_features:
#   means = np.mean(train[num_feature], axis=0)
#   stds = np.std(train[num_feature], axis=0)
#   tada_eta[num_feature] = (tada_eta[num_feature] - means)/stds


# tada_eta.head()

In [ ]:
x_train = np.asarray(train.drop('ATA',1))
y_train = np.asarray(train['ATA'])
x_test = np.asarray(test.drop('ATA',1))
y_test = np.asarray(test['ATA'])
eta_features = [x for i,x in enumerate(tada_eta.columns) if i!=0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print(x_train)

[[  8.32        12.          12.          22.          44.52197739]
 [  3.05        12.          17.          11.           3.98906617]
 [  5.           7.          22.          19.           4.81469879]
 ...
 [ 14.           7.           8.          20.         275.21314119]
 [  7.05        12.          12.           8.           9.17465736]
 [  8.4          7.          14.           7.          14.49508536]]


In [ ]:
print(y_train)

[ 6.13  5.37  6.27 ... 19.75  6.88  4.2 ]


In [ ]:
eta_features

['api_eta', 'month', 'pickup_gu', 'hour', 'distance']

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
import matplotlib.pyplot as plt

#### 원본 : Gradient Boosting 사용

변경 내용 


In [ ]:
indexlist=[]
for i in range(len(tada_eta)):
  if i<12000:
    indexlist.append(-1)
  else:
    indexlist.append(0)

In [ ]:
indexlist[12000]

0

In [ ]:

# Hyperparameter 조합 정의
param_grid = {'max_iter': [i for i in range(60,70,1) ],
              'max_depth': [i for i in range(5,10)],
              'min_samples_leaf': [i for i in range(152,164,1)],
              'learning_rate': [0.04,0.05,0.06],
              'loss': ['squared_error']}

# train set과 test set의 input과 output를 각각 이어 붙여서 X,y를 정의
X = np.concatenate((x_train,x_test), axis=0)
y = np.concatenate((y_train,y_test), axis=0)

# 전체 데이터 X에서 training data에 해당하는 index는 -1, test data에 해당하는 index는 0이 되도록,
# 여기서는 [-1, -1, ... , -1, 0, 0, ... , 0] 같은 형태의 1차원 배열 test fold와 predefined split을 정의
# :1200까지는 train, 1200:부터는 test
test_fold=indexlist
pds = PredefinedSplit(test_fold)
# grid search 모델 정의, 학습 및 model selection
# cv=3 : 3-fold cross validation
grid_search = GridSearchCV(estimator=ensemble.HistGradientBoostingRegressor(random_state=0),
                           param_grid=param_grid,
                           cv=pds, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X, y)
print(grid_search.best_params_)
reg = ensemble.HistGradientBoostingRegressor(**grid_search.best_params_, random_state=0)

Fitting 1 folds for each of 1800 candidates, totalling 1800 fits
{'learning_rate': 0.05, 'loss': 'squared_error', 'max_depth': 8, 'max_iter': 63, 'min_samples_leaf': 160}


In [ ]:
type(x_train)

numpy.ndarray

In [ ]:
reg.fit(x_train, y_train)

HistGradientBoostingRegressor(learning_rate=0.05, max_depth=8, max_iter=63,
                              min_samples_leaf=160, random_state=0)

In [ ]:

mse = mean_squared_error(y_test, reg.predict(x_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

The mean squared error (MSE) on test set: 8.2056
